In [1]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=ZPsd0NK7mv-uOLqc6k0C_BIpKUdJwCWPPwhnW6U725Y&tc=pVLhumTUump3kKz5fJUwRk9WorEac4i0wqlh4dDqt1I&cc=5HN5FtPApWMq8D9r-5dwjoMMSUxYPoGfF3ZIhhl2bkY

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VNt_7Otdl6i3jt4AUdQikgFd3ne-UShXgTb1pvviUIE6yQk8-Pooqk

Successfully saved authorization token.


In [2]:
start_date = "2017-01-01" 
end_date = "2023-01-01" 
start_date = ee.Date(start_date) 
end_date = ee.Date(end_date) 

In [20]:
# get our Sidi Bel Abbesl boundary
aoi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM1_NAME',"Sidi Bel Abbes")).geometry()

In [21]:
NDVI =ee.ImageCollection("MODIS/061/MOD13Q1").select('NDVI')
def calcVCI(image):
    date = image.date()
    history = NDVI \
        .filterDate('2000-01-01', ee.Date.fromYMD(date.get('year').subtract(1), 1, 1)) 
    min = history.min()
    max = history.max()
    vci = image.subtract(min).divide(max.subtract(min))
    return vci.rename('VCI').copyProperties(image, ['system:time_start','system:time_end'])

VCI = NDVI.filterDate(start_date, end_date).map(calcVCI)

In [22]:
LST = ee.ImageCollection("MODIS/061/MOD11A2").select('LST_Day_1km') 
def convert_lst_dates_to_modis_dates(ndvimg):
  start = ndvimg.get('system:time_start')
  end = ndvimg.get('system:time_end')

  composite = LST\
    .filterDate(start, end)\
    .mean() # You need to decide how to combine the images
  return composite\
    .set('system:time_start', start)\
    .set('date', ee.Date(start).format())\
    .set('system:time_end', end)\
    .set('empty', composite.bandNames().size().eq(0))

filteredlst = NDVI.filterDate(start_date, end_date).map(convert_lst_dates_to_modis_dates)

In [23]:
def calcTCI(image):
  date = image.date()

  history = LST.select('LST_Day_1km') \
      .filterDate('2000-02-18', ee.Date.fromYMD(date.get('year').subtract(1), 1, 1)) 
  min = history.min()
  max = history.max()
  tci = max.subtract(image).divide(max.subtract(min))
  return tci.rename('TCI').copyProperties(image, ['system:time_start','system:time_end'])

TCI = filteredlst.filterDate(start_date, end_date).map(calcTCI)


In [24]:
filter = ee.Filter.equals(leftField= 'system:time_start', rightField='system:time_start')
join = ee.Join.saveFirst(matchKey= 'match');

both = ee.ImageCollection(join.apply(VCI , TCI,filter))\
.map(lambda img: img.addBands(img.get('match')).set('date', img.date().format('YYYY_MM_dd')) )
 
VHI= both.map(lambda img:
  img.addBands(
  img.expression('a1/2 + b1/2', {
  "a1": img.select('VCI'),
  "b1": img.select('TCI'),
  }).rename('VHI'))
);

In [19]:
!pip install pandas
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
def create_reduce_region_function(geometry,
                                  reducer=ee.Reducer.mean(),
                                  scale=1000,
                                  crs='EPSG:4326',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4):
  """Creates a region reduction function.

  Creates a region reduction function intended to be used as the input function
  to ee.ImageCollection.map() for reducing pixels intersecting a provided region
  to a statistic for each image in a collection. See ee.Image.reduceRegion()
  documentation for more details.

  Args:
    geometry:
      An ee.Geometry that defines the region over which to reduce data.
    reducer:
      Optional; An ee.Reducer that defines the reduction method.
    scale:
      Optional; A number that defines the nominal scale in meters of the
      projection to work in.
    crs:
      Optional; An ee.Projection or EPSG string ('EPSG:5070') that defines
      the projection to work in.
    bestEffort:
      Optional; A Boolean indicator for whether to use a larger scale if the
      geometry contains too many pixels at the given scale for the operation
      to succeed.
    maxPixels:
      Optional; A number specifying the maximum number of pixels to reduce.
    tileScale:
      Optional; A number representing the scaling factor used to reduce
      aggregation tile size; using a larger tileScale (e.g. 2 or 4) may enable
      computations that run out of memory with the default.

  Returns:
    A function that accepts an ee.Image and reduces it by region, according to
    the provided arguments.
  """

  def reduce_region_function(img):
    """Applies the ee.Image.reduceRegion() method.

    Args:
      img:
        An ee.Image to reduce to a statistic by region.

    Returns:
      An ee.Feature that contains properties representing the image region
      reduction results per band and the image timestamp formatted as
      milliseconds from Unix epoch (included to enable time series plotting).
    """
    
    stat = img.reduceRegion(
        reducer=reducer,
        geometry=geometry,
        scale=scale,
        crs=crs,
        bestEffort=bestEffort,
        maxPixels=maxPixels,
        tileScale=tileScale)
    return ee.Feature(geometry, stat).set({'millis': img.date().millis(), })
  return reduce_region_function

# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

# Function to add date variables to DataFrame.
def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
  df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  return df

reduce_index = create_reduce_region_function(
     geometry=aoi,
     reducer=ee.Reducer.mean(), scale=1000, crs='EPSG:3310')



stat_fc = ee.FeatureCollection(VHI.filterDate(start_date, end_date).map(reduce_index))


data_dict = fc_to_dict(stat_fc).getInfo()
data_df = pd.DataFrame(data_dict)
display(data_df)
print(data_df.dtypes)
data_df = add_date_info(data_df)
data_df = data_df.drop(columns=['millis', 'system:index'])
display(data_df)

,TCI,VCI,VHI,millis,system:index
0,0.798149,0.435502,0.616823,1483228800000,2017_01_01
1,0.827918,0.437044,0.632472,1484611200000,2017_01_17
2,0.687878,0.429227,0.558552,1485993600000,2017_02_02
3,0.598957,0.470779,0.534868,1487376000000,2017_02_18
4,0.476023,0.520016,0.498019,1488758400000,2017_03_06
...,...,...,...,...,...
133,0.442344,0.150476,0.296263,1665878400000,2022_10_16
134,0.522329,0.192309,0.357275,1667260800000,2022_11_01
135,0.655703,0.207999,0.431840,1668643200000,2022_11_17
136,0.726510,0.245962,0.486282,1670025600000,2022_12_03


TCI             float64
VCI             float64
VHI             float64
millis            int64
system:index     object
dtype: object


,TCI,VCI,VHI,Timestamp,Year,Month,Day,DOY
0,0.798149,0.435502,0.616823,2017-01-01,2017,1,1,1
1,0.827918,0.437044,0.632472,2017-01-17,2017,1,17,17
2,0.687878,0.429227,0.558552,2017-02-02,2017,2,2,33
3,0.598957,0.470779,0.534868,2017-02-18,2017,2,18,49
4,0.476023,0.520016,0.498019,2017-03-06,2017,3,6,65
...,...,...,...,...,...,...,...,...
133,0.442344,0.150476,0.296263,2022-10-16,2022,10,16,289
134,0.522329,0.192309,0.357275,2022-11-01,2022,11,1,305
135,0.655703,0.207999,0.431840,2022-11-17,2022,11,17,321
136,0.726510,0.245962,0.486282,2022-12-03,2022,12,3,337
